In [6]:
pd.set_option('display.max_rows', 1000)


In [2]:
import pandas as pd
import os
from pprint import pprint
import requests
import json
from config import api_key
# from geopy.geocoders import Nominatim

In [36]:
url = "https://api.census.gov/data/2018/acs/acs5?get=B06011_001E,B01001_001E,NAME&for=zip%20code%20tabulation%20area&key=" + api_key

response = requests.get(url)
response
response_json = response.json()
# pprint(response_json)



In [37]:
headers = response_json.pop(0)
headers
df = pd.DataFrame(response_json,columns =  headers)
df

,B06011_001E,B01001_001E,NAME,zip code tabulation area
0,33505,17621,ZCTA5 01001,01001
1,16070,30066,ZCTA5 01002,01002
2,4200,11238,ZCTA5 01003,01003
3,35901,4991,ZCTA5 01005,01005
4,42413,14967,ZCTA5 01007,01007
...,...,...,...,...
33115,None,16202,ZCTA5 00907,00907
33116,None,348,ZCTA5 00786,00786
33117,None,69,ZCTA5 00694,00694
33118,None,1568,ZCTA5 00631,00631


In [38]:
ZipZCTA = pd.read_excel('zip_to_zcta_2019.xlsx','ZiptoZCTA_crosswalk',header = 0)
crosswalk = ZipZCTA.loc[ZipZCTA['STATE']=='GA'][['ZIP_CODE','ZCTA']]

In [41]:
census_df = df

In [42]:
census_df = census_df.rename(columns={"B06011_001E":"Median Income", "B01001_001E":"Population", 
                               "NAME":"ZCTA Name","zip code tabulation area":"ZCTA"})
census_df = census_df[['Median Income', 'Population', 'ZCTA']]
census_df.head()

# census_df = census_df.merge(crosswalk, on = 'ZCTA').rename(columns={'ZIP_CODE':'ZIP'})

,Median Income,Population,ZCTA
0,33505,17621,01001
1,16070,30066,01002
2,4200,11238,01003
3,35901,4991,01005
4,42413,14967,01007


In [11]:
# # new data frame with split value columns 
# new = census_df["City/State"].str.split(",", n = 1, expand = True) 
  
# # making separate first name column from new data frame 
# census_df["City"]= new[0] 
  
# # making separate last name column from new data frame 
# census_df["State"]= new[1] 
  
# # Dropping old Name columns 
# census_df.drop(columns =["City/State"], inplace = True)

# census_df


In [50]:
ZipCounty = pd.read_csv("ZipCounty.csv").rename(columns = {"Zip":"ZIP"})
census_df = census_df.rename(columns = {'ZCTA':'ZIP'})
census_df

,Median Income,Population,ZIP
0,33505,17621,01001
1,16070,30066,01002
2,4200,11238,01003
3,35901,4991,01005
4,42413,14967,01007
...,...,...,...
33115,None,16202,00907
33116,None,348,00786
33117,None,69,00694
33118,None,1568,00631


In [46]:
ZipCounty.head(10)

,Unnamed: 0,ZIP,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,County
0,15,31150,Atlanta,GA,33.844371,-84.474050,-5,1,"33.844371,-84.47405",Fulton County
1,16,31646,Saint George,GA,30.518903,-82.080630,-5,1,"30.518903,-82.08063",Washington County
2,17,31826,Shiloh,GA,32.803518,-84.693840,-5,1,"32.803518,-84.69384",Hardin County
3,65,30334,Atlanta,GA,33.702657,-84.439127,-5,1,"33.702657,-84.439127",Fulton County
4,70,31606,Valdosta,GA,30.802411,-83.196240,-5,1,"30.802411,-83.19624",Lowndes County
5,76,31732,Cedar Springs,GA,31.169794,-85.055700,-5,1,"31.169794,-85.0557",Kent County
6,97,30444,East Point,GA,33.696073,-84.450276,-5,1,"33.696073,-84.450276",Fulton County
7,125,30531,Cornelia,GA,34.514215,-83.541180,-5,1,"34.514215,-83.54118",Habersham County
8,202,30746,Sugar Valley,GA,34.574804,-85.024820,-5,1,"34.574804,-85.02482",Gordon County
9,271,30415,Brooklet,GA,32.345600,-81.623390,-5,1,"32.3456,-81.62339",Bulloch County


In [51]:
print(census_df['ZIP'].describe())
print()
print(ZipCounty['ZIP'].describe())

count     33120
unique    33120
top       76832
freq          1
Name: ZIP, dtype: object

count      995.000000
mean     30847.816080
std        661.608695
min      30002.000000
25%      30318.500000
50%      30683.000000
75%      31405.500000
max      39901.000000
Name: ZIP, dtype: float64


In [52]:
census_df['ZIP'] = census_df['ZIP'].astype(float)

In [53]:
ZipEnhanced = ZipCounty.merge(census_df, on = 'ZIP',how = 'left')

In [66]:
ZipEnhanced = ZipEnhanced[['ZIP','City','State','Latitude','Longitude','County','Median Income','Population']]
ZipEnhanced.sort_values(['ZIP']).loc[ZipEnhanced['ZIP'] == 30321]


,ZIP,City,State,Latitude,Longitude,County,Median Income,Population
174,30321,Atlanta,GA,33.844371,-84.47405,Fulton County,NaN,NaN


In [57]:
ZipEnhanced.loc[ZipEnhanced['Population']=='NaN']

,ZIP,City,State,Latitude,Longitude,County,Median Income,Population


In [67]:
ZipEnhanced.to_csv(r"Census_Data_v3.csv")